In [ ]:
library("devtools")
library("Matrix")
library(tgutil)
library(tidyverse)

# load_all("metacell/")
library(metacell)
dir.create("output/scrna_db")
scdb_init("output/scrna_db", force_reinit=T)
scfigs_init("figs/")

In [ ]:
mat_nm = "oc_embexe"

In [ ]:
metadata = read.csv(file = "/net/mraid14/export/tgdata/db/tgdb/mars_runs/stelzer_star/work/220308_01_bak/plates.tsv",stringsAsFactors = F,h = T, sep='\t')
metadata = metadata[metadata$Ref=='oryCun3',]

In [ ]:
mcell_import_multi_mars(mat_nm = mat_nm,
                        dataset_table_fn = paste("../config/oc_samp_key.txt",sep = ""),
                        base_dir = "/net/mraid14/export/tgdata/db/tgdb/mars_runs/stelzer_star/work/220308_01_bak/scdb_oryCun3/output/umi.tab/",
                        patch_cell_name=T,
                        force=TRUE)

In [ ]:
mat = scdb_mat(mat_nm)

In [ ]:
mat@cell_metadata = metadata

In [ ]:
scdb_add_mat(id = 'embs_with_meta', mat=mat)

In [ ]:
meta = fread("config/embryo_meta.txt")
rownames(meta) = meta$Embryo
x = mat@cell_metadata %>% left_join(meta, by='embryo')
rownames(x) = rownames(mat@cell_metadata)
mat@cell_metadata = x
# scdb_add_mat("embs", mat)

In [ ]:
scdb_add_mat("oc_embexe", mat)

In [ ]:
library("reticulate")
library("anndata")
sc = import("scanpy")
mc2 = import("metacells")

In [ ]:
ss3 <- anndata::read_h5ad('cells10_clean_2203.h5ad')
ss3_cells_mc = ss3$obs$metacell
ss3_cells_mc = ss3_cells_mc
cell_names = ss3$obs_names
names(ss3_cells_mc) = cell_names
mc_cols = read.csv('metacell_types-2022-03-27_withexe.csv' ,header = T)[,c('metacell', 'color')]
cell_colors = tibble::enframe(ss3_cells_mc) %>% rename(metacell=value) %>% left_join(mc_cols, by='metacell') %>% rename(mc_col=color)

In [ ]:
mat = scdb_mat("embs5")
message("got ", ncol(mat@mat), " cells")
nms = c(rownames(mat@mat), rownames(mat@ignore_gmat))
#bad_genes = c(grep("^MTRNR", nms, v=T), "NEATL",grep("ERCC", nms,v=T), "ATPASE6", "XIST", "MALATL", "TMSB4X")
# bad_genes = c(grep("^MTRNR", nms, v=T), "NEATL", grep("ERCC", nms,v=T), "ATPASE6", "XIST", "MALATL", "TMSB4X", "T17H1", "ATP6;ATP8;COX3", "GM20594_mm9", "ND3", "COX2")

In [ ]:
mc2_db_dir = "../output/scrna_db"

In [ ]:
# 1. Write the umi matrix into h5ad format (I cleaned the matrix first using MC1, but MC2 can also clean it). This is horribly inefficient (writing 2 files to disk) but I lost myself in scanpy documentation and didn't bother solving it yet.
h5ad_fn = paste(mc2_db_dir , "/oc_cells_with_md.h5ad", sep="")
out_h5ad <- anndata::AnnData(
  X = t(mat@mat),
  obs = mat@cell_metadata
)

In [ ]:
sc$write(adata = out_h5ad, filename = h5ad_fn)